In [ ]:
import geemap
import ee
import json
import os
import requests
import re
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON
import pandas as pd
from datetime import datetime
import numpy as np
from glob import glob
import urllib.request
from matplotlib.pyplot import imshow
from PIL import Image,ImageDraw,ImageFont

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

#initialize GEE using your Google Account
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()
    
    

In [ ]:
#set up GEE map canvas (will be updated with every addLayer() method)
Map = geemap.Map(center=[-27.93186,32.48897],zoom=9)
Map.add_basemap('SATELLITE')
# Map.add_basemap('CartoDB.DarkMatter')

#study area
file_path = './data/boundaries/ramsar_stlucia_bbox.geojson'
with open(file_path) as f:
    coord = json.load(f)['features'][0]['geometry']['coordinates'][0]
    line_geom = ee.Geometry.LineString(coord)
    
    #polygon used for clipping purposes
    poly_geom = ee.Geometry.Polygon(coord)
    
Map.addLayer(ee_object=line_geom, vis_params={'color':'red'}, name="Lesser Isimangaliso Wetland Park")

Map

In [ ]:
#get Sentinel-2 Level-1C collection for 2017


s1_col = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(poly_geom).filterDate('2017-01-01','2017-12-31') \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
    .filter(ee.Filter.eq('instrumentMode', 'IW')).select(['VV','VH'])

s1_col = s1_col.map(lambda x:x.clip(poly_geom))

# #filter products 
s1_col_list = s1_col.toList(s1_col.size())

print('Nr. S1 images found: ', s1_col.size().getInfo())

#remove duplicates based on first occurence unique endtime
s1_products = np.array(s1_col_list.map(lambda x:ee.Image(x).id()).getInfo())
s1_date_un = np.unique(np.array([f.split('_')[4].split('T')[0] for f in s1_products])).tolist()

invalid_dates = np.array(['20170801','20171025','20171106','20171118','20171130','20171212','20171224'])
s1_date_un_valid = np.setdiff1d(np.array(s1_date_un),invalid_dates).tolist()

s1_products_filtered_group=[]

for u_d in s1_date_un_valid:
    group = []
    for prd in s1_products:
        if u_d in prd:
            group.append(prd)
    s1_products_filtered_group.append(group)
    
s1_mos_list = []
for i in range(len(s1_products_filtered_group)):
    group = s1_products_filtered_group[i]
    date = s1_date_un_valid [i]
    s1_mos = s1_col.filter(ee.Filter.inList('system:index', group)).mosaic().clip(poly_geom).set('Date', date)
    s1_mos_list.append(s1_mos)
s1_mos_col = ee.ImageCollection(s1_mos_list)
s1_mos_col_list = s1_mos_col.toList(s1_mos_col.size())

print('Nr. S1 mosaics created: ', s1_mos_col.size().getInfo())

In [ ]:
for i in range(len(s1_mos_list)):
    mos = s1_mos_list[i]
    date = s1_date_un[i]
    Map.addLayer(mos , {'bands':'VH','min': -20, 'max': -7},date) 

In [ ]:
Map.addLayer(ee.Image(s1_col_list.get(-6)), {'bands':'VH','min': -20, 'max': -7}, s1_products[-6]) 

In [ ]:
# export image to drive

#export
aoi = ee_to_geojson(poly_geom)['coordinates'][0]

for i in range(len(s1_date_un_valid)):
    data = ee.Image(s1_mos_col_list.get(i))
    task = ee.batch.Export.image.toDrive(data, 
                                         description='s1_image',
                                         folder='GEE_output',
                                         fileNamePrefix=f's1_mosaic_{s1_date_un_valid[i]}',
                                         crs='EPSG:32736',
                                         scale=10,
                                         maxPixels=1e8,
                                         fileFormat='GeoTIFF',
                                         skipEmptyTiles=True,
                                         region=aoi)
    task.start()

In [ ]:
task.status()